# Committee Spending

*Author: Alan Leggitt (leggitta3@gmail.com)*  
*Date: 2017-06-16*

Visualizes the distribution of total spending across committees.

In [1]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# connect to the database
engine = create_engine(
    "mysql://reader@localhost/campaign_expenditures")

# connect bokeh
output_notebook()

Loading BokehJS ...

In [3]:
# query filings and expenditures to compute total spent per unique committee id
committee_query = """
    SELECT
        fec_committee_id,
        SUM(expenditures.amount) as total_spent
    FROM
        filings
    INNER JOIN
        expenditures
    ON
        expenditures.filing_id = filings.filing_id
    GROUP BY
        fec_committee_id
"""
committee_data = pd.read_sql_query(committee_query, engine)

In [4]:
# compute the log histogram
x = committee_data.total_spent.values
hist, edges = np.histogram(x, bins=np.logspace(0, 10, 10))

# plot log histogram
fig = figure(
    width=600, height=500, toolbar_location=None, x_axis_type="log",
    title="Committee Spending", background_fill_color='oldlace')
fig.title.text_font = "times"
fig.title.text_font_size = "16pt"

fig.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
         line_color='black', fill_color='seagreen')

fig.xaxis.axis_label = "Amount Spent"
fig.xaxis.axis_label_text_font = "times"
fig.xaxis.axis_label_text_font_size = "12pt"

fig.yaxis.axis_label = "Committee Count"
fig.yaxis.axis_label_text_font = "times"
fig.yaxis.axis_label_text_font_size = "12pt"

show(fig)